# Pravděpodobnostní klasifikace (Bayesův klasifikátor)

Úloha zaměřená na implementaci základních pravděpodobnostních metod pro klasifikaci.


## Data
Rozdělení dat uvnitř tříd odpovídá normálnímu rozdělení


In [3]:
import usu
import numpy as np

npzfile = np.load('data/data_11.npz')
npzfile.files


['testData', 'testRef', 'trainData', 'trainRef']

In [4]:
testData = npzfile['testData']
testRef = npzfile['testRef']

trainData = npzfile['trainData']
trainRef = npzfile['trainRef']

nClasses = len(np.unique(trainRef))

trainData.shape,trainRef.shape, testData.shape, testRef.shape


((1900, 2), (1900, 1), (100, 2), (100, 1))

### Výpočet úspěšnosti
$$ accuracy = \frac{\text{počet správně klasifikovaných objektů}}{\text{počet všech klasifikovaných  objektů}} $$

Apriorní pravděpodobnost třídy:

$$ P_{ap}(class) = \frac{\text{počet prvků třídy}}{\text{počet všech prvků všech tříd}} $$



Hustota pravděpodobnosti Gaussova rozdělení pro jednorozměrný příznakový vektor:

$$ f(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp{-\frac{(x-\mu)^2}{2 \sigma^2}} $$

kde $\mu$ je střední hodnota a $\sigma^2$ je rozptyl

Po logoritmizaci dostaneme:
$$ L(x) = -\frac{1}{2} \log(2 \pi \sigma^2) -\frac{(x-\mu)^2}{2 \sigma^2} $$

Pro soubor N vzorků dat:

$$ L(X) = \sum_{i=0}^{N}{ -\frac{1}{2} \log(2 \pi \sigma^2)} -\frac{1}{2}\sum_{i=0}^{N}{\frac{(x_{i}-\mu)^2}{\sigma^2} } $$



Celkově:

$$ L(class, X) = -\frac{1}{2} \sum_{i=0}^{N}{  \log(2 \pi \sigma_{class}^2)} -\sum_{i=0}^{N}{\frac{(x_{i}-\mu_{class})^2}{2 \sigma_{class}^2} } + \log P_{ap}(class)$$

kde $ Z = -\frac{1}{2} \sum_{i=0}^{N}{ \log(2 \pi \sigma_{class}^2)} + \log P(class) = -\frac{N}{2} \log(2 \pi \sigma_{class}^2) + \log P(class) $ je pak pro každou třídu konstanta a je možné si je předpočítat. 



Výpočet pro vícerozměrný příznakový vektor a plnou i diagonální kovariační matici viz. přednášky ....

In [178]:
fixeds = []
means = []
determinants = []
covariances = []

for c in range(nClasses):
    mean = np.mean(trainData[trainRef.flatten() == c], axis=0)
    cov = np.cov(trainData[trainRef.flatten() == c].T)
    ap = len(trainData[trainRef.flatten() == c]) / len(trainData)
    # print("Průměr")
    # print(mean)
    # print("Kovarianční matice")
    # print(cov)
    # print("Apriorní pravděpodobnost")
    # print(ap)
    # print("------------")

    cov = np.diag(np.diag(cov))

    means.append(mean)
    covariances.append(cov)
    det = np.linalg.det(cov)
    determinants.append(det)
    fixed = -1/2*np.log(2*np.pi*det) + np.log(ap)
    fixeds.append(fixed)

predRef = []

for i in range(len(testData)):
    preds = []
    for c in range(nClasses):
        p = -1/2 * (testData[i] - means[c]).T @ np.linalg.inv(covariances[c]) @ (testData[i] - means[c]) + fixeds[c]
        preds.append(p)
    predRef.append(np.argmax(preds))
        


In [163]:
def accuracy(testRef, predRef):

    return (predRef==testRef.T).mean() * 100


In [179]:
print(accuracy(testRef, predRef))

92.0
